In [19]:
import keras
import numpy as np
import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout,Flatten, Lambda,Concatenate
from keras.layers import Conv2D, Activation,AveragePooling2D,MaxPooling2D,BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.preprocessing import image
# from keras import backend as K
import tensorflow.keras.backend as K
import os



2024-05-04 10:56:25.091823: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 10:56:30.037795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/cuda


env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/cuda


In [20]:
def create_base_net(input_shape):
  
# Define input layer
    inputs = Input(shape=input_shape)

    # Convolutional layer 1
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', name='conv1')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 2
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', name='conv2')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 3
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='conv3')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Convolutional layer 4
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='conv4')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Flatten the output for fully connected layers
    x = Flatten()(x)

    # Fully Connected layer 1
    x = Dense(1000, activation='relu', name='fc1')(x)

    # Fully Connected layer 2
    fc2_output = Dense(128, activation='sigmoid', name='fc2')(x)

    # Output layer (features from FC layer 2)
    features = fc2_output

    # Create model
    model = Model(inputs=inputs, outputs=features)

    # Summary of the model
    # model.summary()
  
    return model

def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


In [21]:
input_shape=(300,300,1)

In [4]:
# def classify(output,threshold=0.5):
#     return K.cast(K.greater(output, threshold), K.floatx())

In [22]:
import os
import random
directory = "./database"
filenames = os.listdir(directory)

# Initialize an empty list to store split filenames
split_filenames = []
X=[]
X1=[]
y=[]
hand={}
lhand={}
rhand={}
# Split each filename by underscore and append to the list
for filename in filenames:
    split_filename = filename.split('_')

    # Rename files by removing 850(Which represents wavelength)
    # split_filename.pop(2)
    # new_filename = "_".join(split_filename)
    # os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
    if split_filename[0] in hand:
        hand[split_filename[0]].append(filename)
        if split_filename[1]=='l':
            if split_filename[0] in lhand:
                lhand[split_filename[0]].append(filename)
            else:
                lhand[split_filename[0]]=[filename]
        else:
            if split_filename[0] in rhand:
                rhand[split_filename[0]].append(filename)
            else:
                rhand[split_filename[0]]=[filename]
            # rhand[split_filename[0]].append(filename)
    else:
        hand[split_filename[0]]=[filename]
        if split_filename[1]=='l':
            lhand[split_filename[0]]=[filename]
        else:
            rhand[split_filename[0]]=[filename]
    
    split_filenames.append(split_filename)

# print(lhand.values())
# print(hand)
# print(lhand)
# print(rhand)
count=0
# for a,b,c,d in zip(list(lhand.values())[0],list(rhand.values())[0],list(lhand.values())[1],list(rhand.values())[1]):
#     print("You printing",a,b,c,d)
#     if count
#     X1.append([a,b,c,d])
l1,r1,l2,r2=list(lhand.values())[0],list(rhand.values())[0],list(lhand.values())[1],list(rhand.values())[1]
for count in range(len(l1)):
    # rand_pair=random.randint(1,2)
    # if rand_pair==1:
    if count<=1:
        X1.append([l1[count],r1[count],l2[count],r2[count]])
        y.append(0)
    else:
        r=random.sample(range(0,6),2)
        print(r)
        X1.append([l1[r[0]],r1[r[0]],l1[r[1]],l1[r[1]]])
        y.append(1)
    # print(rand_pair)
    
    # X1.append()
# print(split_filenames)
print(X1)
print(y)

[1, 2]
[4, 2]
[2, 1]
[0, 3]
[['002_l_03.jpg', '002_r_04.jpg', '001_l_01.jpg', '001_r_06.jpg'], ['002_l_05.jpg', '002_r_05.jpg', '001_l_05.jpg', '001_r_03.jpg'], ['002_l_05.jpg', '002_r_05.jpg', '002_l_02.jpg', '002_l_02.jpg'], ['002_l_06.jpg', '002_r_02.jpg', '002_l_02.jpg', '002_l_02.jpg'], ['002_l_02.jpg', '002_r_01.jpg', '002_l_05.jpg', '002_l_05.jpg'], ['002_l_03.jpg', '002_r_04.jpg', '002_l_04.jpg', '002_l_04.jpg']]
[0, 0, 1, 1, 1, 1]


In [23]:
train_image = []
X2=[]
X3=[]
X4=[]
X5=[]
for i in X1:
    train_subset=[]
    c=0
    for j in i:
        # print(c)
        img = image.load_img('./database/'+j, target_size=(300,300), color_mode="grayscale")
        img = image.img_to_array(img)
        img = img/255
        train_subset.append(img)
        if c==0:
            X2.append(img)
        elif c==1:
            X3.append(img)
        elif c==2:
            X4.append(img)
        elif c==3:
            X5.append(img)
        c+=1
        
    # train_image.append(train_subset)
# X = np.array(train_image)
X2=np.array(X2)
X3=np.array(X3)
X4=np.array(X4)
X5=np.array(X5)
y=np.array(y)
print(np.shape(X2))
print(type(X2))

(6, 300, 300, 1)
<class 'numpy.ndarray'>


In [24]:

base_network = create_base_net((300,300,1))
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
input_c = Input(shape=input_shape)
input_d = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)
concatenated_features = Concatenate()([processed_a, processed_b])

processed_c = base_network(input_c)
processed_d = base_network(input_d)
concatenated_outputs = Concatenate()([processed_c, processed_d])

# distance = Lambda(euclid_dis,
#                   output_shape=eucl_dist_output_shape)([processed_a, processed_b])
difference = Lambda(lambda x: abs(x[0] - x[1]), name='difference')([concatenated_features, concatenated_outputs])

probability = Dense(1, activation='sigmoid', name='probability')(difference)
# thresholded_probability = Lambda(lambda x: K.cast(K.greater(x, 0.5), K.floatx()), name='thresholded_probability')(probability)

model = Model([input_a, input_b,input_c,input_d], outputs=probability)


# predictions = model.predict([input_data_a, input_data_b, input_data_c, input_data_d])
# classifications = classify(predictions)
model.summary()

2024-05-04 10:56:36.259520: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-04 10:56:36.514228: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-04 10:56:36.514392: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, 128)       │ 20,977,576 │ input_layer_1[0]… │
│ (Functional)        │                   │            │ input_layer_2[0]… │
│                     │                   │            │ input_layer_3[0]… │
│                     │                   │            │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ functional_1[0][… │
│ (Concatenate)       │                   │            │ functional_1[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ functional_1[2][… │
│ (Concatenate)       │                   │            │ functional_1[3][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ difference (Lambda) │ (None, 256)       │          0 │ concatenate[0][0… │
│                     │                   │            │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ probability (Dense) │ (None, 1)         │        257 │ difference[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 20,977,833 (80.02 MB)

 Trainable params: 20,977,321 (80.02 MB)

 Non-trainable params: 512 (2.00 KB)

In [25]:
adam = Adam()
model.compile(loss=contrastive_loss, optimizer=adam, metrics=[accuracy])
model.fit([X2,X3,X4,X5], y,
          batch_size=128,
          epochs=5,)
        #   validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

Epoch 1/5


I0000 00:00:1714800403.250331    5281 service.cc:145] XLA service 0x586ffe76ca80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714800403.250444    5281 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 with Max-Q Design, Compute Capability 7.5
2024-05-04 10:56:44.167741: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-04 10:56:45.656183: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.6667 - loss: 0.2308

I0000 00:00:1714800417.559664    5281 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.6667 - loss: 0.2308
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.6667 - loss: 0.1442
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step - accuracy: 0.5556 - loss: 0.0879
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.5556 - loss: 0.0593
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.5556 - loss: 0.0175


In [16]:
%%bash

# Set XLA_FLAGS environment variable
XLA_FLAGS="--xla_gpu_cuda_data_dir=/opt/cuda"
python -c "import os; print(os.environ.get('XLA_FLAGS', 'Not set'))"


Not set


In [1]:
!python -c "import os; print(os.environ.get('XLA_FLAGS', 'Not set'))"

Not set


In [2]:
%%bash

export PYENV_ROOT="$HOME/.pyenv"
[[ -d $PYENV_ROOT/bin ]] && export PATH="$PYENV_ROOT/bin:$PATH"
eval "$(pyenv init --path)"

In [3]:
!python --version

Python 3.11.0


env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/opt/cuda
